## Import librairies

In [2]:
import spacy
import spacy_transformers
from spacy import displacy
import fr_dep_news_trf
nlp = fr_dep_news_trf.load()
spacy.require_cpu() 

from docx.api import Document

import pandas as pd
import random
import glob
import subprocess
import re
import os
import shutil
import itertools
from collections import Counter
from pathlib import Path

/home/gcortal/anaconda3/envs/M2_syntactic_analysis/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Functions

In [3]:
def MAPA(text):
    """Disidentify a text using MAPA tool."""
    return text

In [4]:
def tokenize(text):
    """Return a tokenized text."""
    doc = nlp(text)
    return [token for token in doc]

In [5]:
def doc_to_docx():
    """Convert from doc to docx extension."""
    for doc in glob.iglob("*.doc"):
        subprocess.call(['soffice', '--convert-to', 'docx', doc])

In [6]:
def fast_scandir(dirname):
    """Return subfolders of a directory (recursive)."""
    subfolders= [f.path for f in os.scandir(dirname) if f.is_dir()]
    for dirname in list(subfolders):
        subfolders.extend(fast_scandir(dirname))
    return subfolders

In [7]:
def deidentify(filename):
    """Disidentify a docx document."""
    document = Document(filename)
    
    for table in document.tables:
        for row in table.rows:
            for cell in row.cells:
                for paragraph in cell.paragraphs:
                    paragraph.text = MAPA(paragraph.text)

    for paragraph in document.paragraphs:
        paragraph.text = MAPA(paragraph.text)

    for section in document.sections:
        section.header.is_linked_to_previous = True
        section.footer.is_linked_to_previous = True
    
    document.save(filename)

In [8]:
def get_nb_paragraph(document):
    """Return number of paragraphs and text length."""
    try:
        count = 0
        text_length = 0
        for p in document.paragraphs:
            if len(p.text) != 0:
                count = count + 1
                text_length = text_length + len(p.text)
                #print(p.text)
        #print(count)
        #print(text_length)
    except:
        return 0, 0
    return count, text_length

In [9]:
def table_to_df(document):
    """Convert the first table of a document to a DataFrame."""
    table = document.tables[0]

    data = []

    keys = None
    for i, row in enumerate(table.rows):
        text = (cell.text for cell in row.cells)

        if i == 0:
            keys = tuple(text)
            continue
        row_data = dict(zip(keys, text))
        data.append(row_data)


    return pd.DataFrame(data)

In [10]:
def get_stats(document):
    """Return number of paragraphs, tables, rows, columns, sections."""
    try:
        nb_table = len(document.tables)
        df = table_to_df(document)
        nb_row = df.shape[0]
        nb_col = df.shape[1]
    except:
        nb_table = 0
        nb_row = 0
        nb_col = 0
    try:
        nb_section = len(document.sections)
    except:
        nb_section = 0
    nb_paragraph, text_length = get_nb_paragraph(document)
    return [nb_table, nb_row, nb_col, nb_section, nb_paragraph]

In [11]:
def tokenize_document(document):
    """Return table and paragraph tokens."""
    table_tokens = list()
    paragraph_tokens = list()

    for table in document.tables:
        for row in table.rows:
            for cell in row.cells:
                for paragraph in cell.paragraphs:
                    table_tokens.append(tokenize(paragraph.text))

    for paragraph in document.paragraphs:
        paragraph_tokens.append(tokenize(paragraph.text))

    return [table_tokens, paragraph_tokens]

In [12]:
def get_text(document):
    """Return text in the last column of document tables (corresponding to author answers)."""
    sentence_l = list()
    
    try:

    
        for table in document.tables:
            data = list()

            keys = None
            for i, row in enumerate(table.rows):
                text = (cell.text for cell in row.cells)

                if i == 0:
                    keys = tuple(text)
                    continue
                row_data = dict(zip(keys, text))
                data.append(row_data)

            df = pd.DataFrame(data)

            try:
                sentence = df.iloc[: , -1]

                sentence = df.columns[-1] + " " + " ".join(list(df.iloc[: , -1]))
                sentence = " ".join(sentence.split())
                sentence_l.append(sentence)
            except:
                sentence_l.append("")
    except:
        return ""
    
    #return " ".join(sentence_l)
    return sentence_l

## Extract and disidentify FERA grids

In [ ]:
for folder in fast_scandir("/home/gcortal/Documents/M2/grille"):
    os.chdir(folder)
    doc_to_docx()
    
i = 0
for folder in fast_scandir("/home/gcortal/Documents/M2/grille"):
    os.chdir(folder)
    for doc in glob.iglob("*.docx"):
        i = i + 1
        shutil.move(doc, f"/home/gcortal/Documents/M2/grille_processed/{i}.docx")
        
for doc in glob.iglob("/home/gcortal/Documents/M2/grille_processed/*.docx"):
    print(doc)
    try:
        deidentify(doc)
    except:
        print("Format non supporté")

## CELLS FOR TESTING

In [47]:
path = Path(f'/home/gcortal/Documents/M2/grille_processed/{random.randint(1,598)}.docx')
document = Document(path)
#document = Document('/home/gcortal/Documents/M2/grille_modele/2013/EMOTIONS-nouvelle-nov-2013.docx')
table = document.tables[0]

data = []

keys = None
for i, row in enumerate(table.rows):
    text = (cell.text for cell in row.cells)

    if i == 0:
        keys = tuple(text)
        continue
    row_data = dict(zip(keys, text))
    data.append(row_data)
#print (data)


df = pd.DataFrame(data)

In [48]:
df

,FAITS,EXTÉRIEURS,"\nLe contexte : lieu, temps, personnes, situation, actions…","Je suis au cabinet, seul, nous sommes lundi et demain, je retourne à l’école pour donner cours après 15 jours de vacances scolaires.\nJ’appréhende de me retrouver face au coordinateur clinique avec qui j’ai eu une altercation avant les vacances au sujet de mon retard de 20 min un samedi matin."
0,FAITS,INTÉRIEURS,Mes ressentis physiques :\nlocalisations et pr...,J’ai une sensation d’oppression dans le thorax...
1,FAITS,INTÉRIEURS,"Le film de mes pensées :\n images, sons, mots …",A chaque fois que je pense à cette situation e...
2,,Vérification des faits,Contexte et ressentis \nsont-ils exprimés fact...,Contexte : exprimé factuellement\nRessentis : ...
3,,Identification de l’émotion,Une ou plusieurs ?\nLaquelle est traitée ici ?,Peur/ dégoût
4,RAISONS,colère/peur,Quel «territoire» attaqué?\nPar qui/quoi ?,Le territoire attaqué est mon territoire de co...
5,RAISONS,Vérifications\ncolère/peur,Le territoire est-il à moi ?\nL’attaque est ex...,"Il s’agit bien de mon territoire, l’attaque es..."
6,,,Ce que m’apporte cette analyse au moment de l’...,"Rien, car je suis dans l’attente et dans l’exp..."
7,ACTIONS,INTÉRIEURES,Mes actions\n intérieures \n qui ont permis de...,L’émotion n’est pas diminuée dans le sens où i...
8,ACTIONS,EXTÉRIEURES,Mes actions\nextérieures \n qui ont permis de ...,Le seul moyen de stopper l’attaque est la fuit...


In [52]:
#print(df.iloc[:, :-1].to_latex(index=False))  

\begin{tabular}{lll}
\toprule
  FAITS &                  EXTÉRIEURS & \textbackslash nLe contexte : lieu, temps, personnes, situation, actions… \\
\midrule
  FAITS &                  INTÉRIEURS & Mes ressentis physiques :\textbackslash nlocalisations et pré... \\
  FAITS &                  INTÉRIEURS &    Le film de mes pensées :\textbackslash n images, sons, mots … \\
        &     Vérification des faits  & Contexte et ressentis \textbackslash nsont-ils exprimés factu... \\
        & Identification de l’émotion &     Une ou plusieurs ?\textbackslash nLaquelle est traitée ici ? \\
RAISONS &                 colère/peur &         Quel «territoire» attaqué?\textbackslash nPar qui/quoi ? \\
RAISONS &  Vérifications\textbackslash ncolère/peur & Le territoire est-il à moi ?\textbackslash nL’attaque est ext... \\
        &                             & Ce que m’apporte cette analyse au moment de l’é... \\
ACTIONS &                 INTÉRIEURES & Mes actions\textbackslash n intérieures \textbac

In [53]:
#print(df.iloc[:, -1].to_latex(index=False))  

\begin{tabular}{l}
\toprule
Je suis au cabinet, seul, nous sommes lundi et demain, je retourne à l’école pour donner cours après 15 jours de vacances scolaires.\textbackslash nJ’appréhende de me retrouver face au coordinateur clinique avec qui j’ai eu une altercation avant les vacances au sujet de mon retard de 20 min un samedi matin. \\
\midrule
J’ai une sensation d’oppression dans le thorax,... \\
A chaque fois que je pense à cette situation et... \\
Contexte : exprimé factuellement\textbackslash nRessentis : f... \\
                                     Peur/ dégoût  \\
Le territoire attaqué est mon territoire de con... \\
Il s’agit bien de mon territoire, l’attaque est... \\
Rien, car je suis dans l’attente et dans l’expe... \\
L’émotion n’est pas diminuée dans le sens où il... \\
Le seul moyen de stopper l’attaque est la fuite... \\
\bottomrule
\end{tabular}



In [13]:
def get_category(text):
    
    text = re.sub("\n", " ", text)
    text = " ".join(text.split())
    
    text_origin = text
    
    text = text.lower()

    if re.search(r"(^| )fait", text) and not re.search("ACTION", text_origin) and not re.search("vérification", text) and not re.search("raison", text):

        if re.search("extérieur", text):
            
            return "fait extérieur"

        elif re.search("intérieur", text):

            if re.search("physique", text) or re.search("(ressenti|ressens)", text):
                
                return "fait intérieur ressentis"

            elif re.search(r"(^| )(pensée|pense)", text):
                
                return "fait intérieur pensées"

            else:
                
                return "fait intérieur"

        else:
            
            #return "fait"
            return get_category_flexible(text_origin)

    elif re.search("raison", text) and not re.search(r"(^| )fait", text) and not re.search("vérification", text) and not re.search(r"(^| )action", text):

        if re.search("identification", text) and re.search("émotion", text):
            
            return "identification émotion"
        
        elif re.search("territoire", text):
            
            return "territoire (colère/peur)"

        else:
            
            return "territoire (tristesse/joie)"

    elif re.search(r"(^| )action", text) and not re.search("FAIT", text_origin):

        if re.search("extérieur", text):
            
            return "action extérieur"

        elif re.search("intérieur", text):
            
            return "action intérieur"

        elif re.search("test", text):
            
            return "action test"

        else:
            
            return "action"

    elif re.search("vérification", text):

        if re.search("fait", text):
            
            return "vérification fait"

        elif re.search("(colère|tristesse|joie|peur)", text):
            
            return "vérification émotion"
            
        else:
            
            return "vérification"
            
    else:
        
        return get_category_flexible(text_origin)
        

In [14]:
def get_category_flexible(text):
    
    text_origin = text
    
    text = text.lower()
    

    if re.search("identification", text) and re.search("émotion", text):
        
        return "identification émotion"
        
    elif re.search("physique", text) or re.search("(ressenti|ressens)", text):
        
        return "fait intérieur ressentis"

    elif re.search(r"(^| )(pensée|pense)", text):
        
        return "fait intérieur pensées"
    
    elif re.search(r"(^| )(factuel|contexte)", text):
        
        return "fait extérieur"
        
    elif re.search("besoin", text):
        
        return "besoin"
        
    elif re.search("si (colère|peur)", text):
        
        return "territoire (colère/peur)"
    
    elif re.search("(si (tristesse|joie)|objet aimé)", text):
    
        return "territoire (tristesse/joie)"
        
    elif re.search("territoire", text):

        return "territoire"
    
    elif text == "":
        
        return "identification émotion"
    
    elif re.search("lieu", text) and re.search("temps", text):
        
        return "fait extérieur"
    
    #En relisant les faits, l’autre voulait-il m’attaquer ou est-ce un interprétation ? 
    
    elif re.search("interprétation", text):
        
        return "interprétation"
    
    elif re.search("m’apporte", text):
        
        return "apport analyse"
    
    elif re.search("repères mentaux", text):
        
        return "repères mentaux"
    
    elif re.search("repères corporels", text):
        
        return "repères corporels"

    else:

        return "rien"
        

In [15]:
def extract_table(table):

    data = []

    keys = None
    for i, row in enumerate(table.rows):
        text = (cell.text for cell in row.cells)

        if i == 0:
            keys = tuple(text)
            continue
        row_data = dict(zip(keys, text))
        data.append(row_data)


    return pd.DataFrame(data)

In [16]:
def normalize(df):
    
    ndata_l = list()

    ndata_l.append([" ".join(list(df.columns[:-1])), df.columns[-1:][0]])

    for i in range(len(df)):
        ndata_l.append([" ".join(list(df.iloc[i, :-1])), df.iloc[i, -1]])
        
    return ndata_l , pd.DataFrame(ndata_l, columns=["category", "answers"])

In [17]:
def extract_label(data_l):
    
    category_l = list()

    for text in data_l:
        
        category_l.append(get_category(text[0]))
    
    for i, data in enumerate(data_l):
        data.append(category_l[i])
    
    return data_l, pd.DataFrame(data_l, columns=["category", "answers", "label"])
    

In [18]:
def categorize(path):

    document = Document(path)
    
    df_total = pd.DataFrame(columns=["category", "answers", "label"])
    
    for i, table in enumerate(document.tables):

        df = extract_table(table)

        ndata_l, df_normalized = normalize(df)
        
        ndata_l, df_label = extract_label(ndata_l)

        df_label['grille'] = i + 1
        df_total = df_total.append(df_label, ignore_index=True)

    df_total['name'] = path.name
    
    return df_total

In [19]:
df_total = pd.DataFrame(columns=["category", "answers", "label", "grille", "name"])
docs = glob.iglob('/home/gcortal/Documents/M2/grille_processed/*.docx')
for doc in docs:
    path = Path(doc)
    try:
        df_doc = categorize(path)
    
        df_total = df_total.append(df_doc, ignore_index = True)
    except:
        print(doc)

/home/gcortal/Documents/M2/grille_processed/597.docx
/home/gcortal/Documents/M2/grille_processed/526.docx
/home/gcortal/Documents/M2/grille_processed/576.docx
/home/gcortal/Documents/M2/grille_processed/561.docx
/home/gcortal/Documents/M2/grille_processed/559.docx
/home/gcortal/Documents/M2/grille_processed/193.docx
/home/gcortal/Documents/M2/grille_processed/570.docx
/home/gcortal/Documents/M2/grille_processed/587.docx
/home/gcortal/Documents/M2/grille_processed/535.docx
/home/gcortal/Documents/M2/grille_processed/35.docx
/home/gcortal/Documents/M2/grille_processed/590.docx
/home/gcortal/Documents/M2/grille_processed/555.docx
/home/gcortal/Documents/M2/grille_processed/575.docx
/home/gcortal/Documents/M2/grille_processed/549.docx
/home/gcortal/Documents/M2/grille_processed/596.docx
/home/gcortal/Documents/M2/grille_processed/545.docx
/home/gcortal/Documents/M2/grille_processed/532.docx
/home/gcortal/Documents/M2/grille_processed/592.docx
/home/gcortal/Documents/M2/grille_processed/563

In [20]:
df_total

,category,answers,label,grille,name
0,"FAITS EXTÉRIEURS Le contexte : lieu, temps, pe...","Aéroport lors du passage de la sécurité, la pe...",fait extérieur,1.0,142.docx
1,FAITS INTÉRIEURS Mes ressentis physiques :\nlo...,"Serre les dents, les mains et temperature qui ...",fait intérieur ressentis,1.0,142.docx
2,FAITS INTÉRIEURS Le film de mes pensées :\nima...,mots : Il est con ou il fait exprès (pensées a...,fait intérieur pensées,1.0,142.docx
3,RAISONS colère Quel «territoire» attaqué?\n\nP...,territoire attaqué : pensées tes affaires\npa...,territoire (colère/peur),1.0,142.docx
4,RAISONS Vérifications\ncolère Le territoire es...,le territoire est à moi et l’attaque ext.,vérification émotion,1.0,142.docx
...,...,...,...,...,...
11003,Identification de l’émotion Une ou plusieurs ...,la peur. Mais plusieurs peurs : une peur pour...,identification émotion,4.0,276.docx
11004,RAISONS Si colère/peur Quel «territoire» attaq...,,territoire (colère/peur),4.0,276.docx
11005,Vérifications\ncolère/peur Le territoire est-...,"Non, ce n’est pas mon corps qui est attaqué. C...",vérification émotion,4.0,276.docx
11006,ACTIONS INTÉRIEURES,\nJe me persuade de convaincre d’autres client...,action intérieur,4.0,276.docx


In [86]:
df_total.query("label == 'apport analyse'")

,category,answers,label,grille,name
149,Ce que m’apporte cette analyse au moment de ...,"Sans la grille FRA, j’aurais sans doute exprim...",apport analyse,1.0,297.docx
203,Ce que m’apporte cette analyse au moment de ...,"En réalisant l’émotion qui me submerge, j’ai p...",apport analyse,1.0,249.docx
248,"Ce que m’apporte, pendant l’émotion, cette rel...",Un sourire ! je me dis que je ne suis pas enco...,apport analyse,1.0,356.docx
257,Ce que m’apporte cette analyse au moment de ...,Cette analyse me permet de relativiser la situ...,apport analyse,1.0,124.docx
267,Ce que m’apporte cette analyse au moment de ...,Je fais cette analyse a posteriori car je n’ai...,apport analyse,2.0,124.docx
...,...,...,...,...,...
10852,"Ce que m’apporte, pendant l’émotion, cette rel...","Je me suis calmée rapidement, soulagement.",apport analyse,1.0,361.docx
10876,Ce que m’apporte cette analyse au moment de ...,,apport analyse,1.0,224.docx
10886,Ce que m’apporte cette analyse au moment de ...,Cette analyse m’a permis d’éviter une joute ve...,apport analyse,1.0,213.docx
10921,Ce que m’apporte cette analyse au moment de ...,"J’ai très vite analysé ma colère, ce qui m’a é...",apport analyse,1.0,371.docx


In [29]:
df_total.query("category == ''")

,category,answers,label,grille,name
41,,Colère,identification émotion,1.0,168.docx
50,,Peur,identification émotion,2.0,168.docx
224,,Colère,identification émotion,1.0,91.docx
233,,Peur,identification émotion,2.0,91.docx
241,,Emotion (colère),identification émotion,1.0,356.docx
...,...,...,...,...,...
10758,,Colère,identification émotion,1.0,77.docx
10767,,Peur,identification émotion,2.0,77.docx
10845,,Emotion (colère),identification émotion,1.0,361.docx
10934,,Colère,identification émotion,1.0,147.docx


In [21]:
df_total.query("label == 'rien' & answers != ''")

,category,answers,label,grille,name
28,+ Etat final :,Mais moi je me dis alors que j'en ai fait trop...,rien,1.0,432.docx
746,L’objet dont je suis séparé ?\nPar qui/quoi ?,"Je suis séparé d’un bon repas (complet), par m...",rien,3.0,313.docx
994,FAITS Mon commentaire,sans doute de façon trop peu discrète pour ell...,rien,1.0,498.docx
1353,AC\nTIONS INTÉRIEURES,Cette case sera faite pendant le module 2,rien,1.0,519.docx
1355,AC\nTIONS EXTÉRIEURES,Cette case sera faite pendant le module 2,rien,1.0,519.docx
...,...,...,...,...,...
9820,+ Etat final :,Mais moi je me dis alors que j'en ai fait trop...,rien,1.0,440.docx
10167,"FAITS EXTÉRIEURS \nAu travail, des clients son...",Tous pressés car le travail est en retard et l...,rien,1.0,136.docx
10380,FAITS Mon commentaire,sans doute de façon trop peu discrète pour ell...,rien,1.0,509.docx
10432,"EXTÉRIEURES Demande possible, …",La prochaine fois je prouverai que ma solution...,rien,1.0,150.docx


In [22]:
len(set(list(df_total.query("label == 'rien' & answers != ''")['name'])))

39

In [26]:
df_total['label'] = df_total['label'].astype("category")

In [28]:
df_total['label'].describe()

count              11008
unique                21
top       fait extérieur
freq                1250
Name: label, dtype: object

In [77]:
pd.value_counts(df_total['label'])

fait extérieur                 1250
fait intérieur pensées         1214
fait intérieur ressentis       1213
identification émotion         1140
vérification émotion           1041
action extérieur                973
action intérieur                877
vérification fait               796
territoire (colère/peur)        771
territoire (tristesse/joie)     513
action test                     408
besoin                          219
territoire                      126
apport analyse                  124
interprétation                  123
rien                            102
repères corporels                47
repères mentaux                  47
action                           21
fait intérieur                    2
vérification                      1
Name: label, dtype: int64

In [78]:
print(pd.value_counts(df_total['label']).to_latex(index=True))

\begin{tabular}{lr}
\toprule
{} &  label \\
\midrule
fait extérieur              &   1250 \\
fait intérieur pensées      &   1214 \\
fait intérieur ressentis    &   1213 \\
identification émotion      &   1140 \\
vérification émotion        &   1041 \\
action extérieur            &    973 \\
action intérieur            &    877 \\
vérification fait           &    796 \\
territoire (colère/peur)    &    771 \\
territoire (tristesse/joie) &    513 \\
action test                 &    408 \\
besoin                      &    219 \\
territoire                  &    126 \\
apport analyse              &    124 \\
interprétation              &    123 \\
rien                        &    102 \\
repères corporels           &     47 \\
repères mentaux             &     47 \\
action                      &     21 \\
fait intérieur              &      2 \\
vérification                &      1 \\
\bottomrule
\end{tabular}



In [37]:
df_total['label'].values.codes

array([ 6,  9,  8, ..., 20,  2,  1], dtype=int8)

In [44]:
df_total.iloc[46]['category'] #En relisant les faits, l’autre voulait-il m’attaquer ou est-ce un interprétation\xa0?

'En relisant les faits, l’autre voulait-il m’attaquer ou est-ce un interprétation\xa0?'

In [45]:
df_total.iloc[149]['category'] #Ce que m’apporte cette analyse au moment de l’émotion

'  Ce que m’apporte cette analyse au moment de l’émotion '

### Steps

In [59]:
ndata_l, df_normalized = normalize(df)

In [60]:
df_normalized

,category,answers
0,"FAITS EXTÉRIEURS \nLe contexte : lieu, temps, ...","Je suis au cabinet, seul, nous sommes lundi et..."
1,FAITS INTÉRIEURS Mes ressentis physiques :\nlo...,J’ai une sensation d’oppression dans le thorax...
2,FAITS INTÉRIEURS Le film de mes pensées :\n im...,A chaque fois que je pense à cette situation e...
3,Vérification des faits Contexte et ressentis...,Contexte : exprimé factuellement\nRessentis : ...
4,Identification de l’émotion Une ou plusieurs ...,Peur/ dégoût
5,RAISONS colère/peur Quel «territoire» attaqué?...,Le territoire attaqué est mon territoire de co...
6,RAISONS Vérifications\ncolère/peur Le territoi...,"Il s’agit bien de mon territoire, l’attaque es..."
7,Ce que m’apporte cette analyse au moment de ...,"Rien, car je suis dans l’attente et dans l’exp..."
8,ACTIONS INTÉRIEURES Mes actions\n intérieures ...,L’émotion n’est pas diminuée dans le sens où i...
9,ACTIONS EXTÉRIEURES Mes actions\nextérieures \...,Le seul moyen de stopper l’attaque est la fuit...


In [63]:
print(df_normalized.to_latex(index=False))

\begin{tabular}{ll}
\toprule
                                          category &                                            answers \\
\midrule
FAITS EXTÉRIEURS \textbackslash nLe contexte : lieu, temps, p... & Je suis au cabinet, seul, nous sommes lundi et ... \\
FAITS INTÉRIEURS Mes ressentis physiques :\textbackslash nloc... & J’ai une sensation d’oppression dans le thorax,... \\
FAITS INTÉRIEURS Le film de mes pensées :\textbackslash n ima... & A chaque fois que je pense à cette situation et... \\
 Vérification des faits  Contexte et ressentis ... & Contexte : exprimé factuellement\textbackslash nRessentis : f... \\
 Identification de l’émotion Une ou plusieurs ?... &                                      Peur/ dégoût  \\
RAISONS colère/peur Quel «territoire» attaqué?\textbackslash ... & Le territoire attaqué est mon territoire de con... \\
RAISONS Vérifications\textbackslash ncolère/peur Le territoir... & Il s’agit bien de mon territoire, l’attaque est... \\
  Ce que m’apporte cet

In [61]:
ndata_l, df_label = extract_label(ndata_l)

In [72]:
print(df_label.to_latex(index=False))

\begin{tabular}{ll}
\toprule
                   label &                                            answers \\
\midrule
          fait extérieur & Je suis au cabinet, seul, nous sommes lundi et ... \\
fait intérieur ressentis & J’ai une sensation d’oppression dans le thorax,... \\
  fait intérieur pensées & A chaque fois que je pense à cette situation et... \\
       vérification fait & Contexte : exprimé factuellement\textbackslash nRessentis : f... \\
  identification émotion &                                      Peur/ dégoût  \\
territoire (colère/peur) & Le territoire attaqué est mon territoire de con... \\
    vérification émotion & Il s’agit bien de mon territoire, l’attaque est... \\
          apport analyse & Rien, car je suis dans l’attente et dans l’expe... \\
        action intérieur & L’émotion n’est pas diminuée dans le sens où il... \\
        action extérieur & Le seul moyen de stopper l’attaque est la fuite... \\
\bottomrule
\end{tabular}



In [69]:
columns_titles = ["label","answers"]
df_label=df_label.reindex(columns=columns_titles)

In [71]:
df_label

,label,answers
0,fait extérieur,"Je suis au cabinet, seul, nous sommes lundi et..."
1,fait intérieur ressentis,J’ai une sensation d’oppression dans le thorax...
2,fait intérieur pensées,A chaque fois que je pense à cette situation e...
3,vérification fait,Contexte : exprimé factuellement\nRessentis : ...
4,identification émotion,Peur/ dégoût
5,territoire (colère/peur),Le territoire attaqué est mon territoire de co...
6,vérification émotion,"Il s’agit bien de mon territoire, l’attaque es..."
7,apport analyse,"Rien, car je suis dans l’attente et dans l’exp..."
8,action intérieur,L’émotion n’est pas diminuée dans le sens où i...
9,action extérieur,Le seul moyen de stopper l’attaque est la fuit...


In [ ]:
get_stats(document)

## Create a DataFrame containing statistics and text

In [80]:
data = list()
docs = glob.iglob('/home/gcortal/Documents/M2/grille_processed/*.docx')
for doc in docs:
    print(doc)
    path = Path(doc)
    document = Document(doc)
    text = get_text(document)
    data.append([path.name] + get_stats(document) + [text])

/home/gcortal/Documents/M2/grille_processed/142.docx
/home/gcortal/Documents/M2/grille_processed/432.docx
/home/gcortal/Documents/M2/grille_processed/168.docx
/home/gcortal/Documents/M2/grille_processed/558.docx
/home/gcortal/Documents/M2/grille_processed/481.docx
/home/gcortal/Documents/M2/grille_processed/504.docx
/home/gcortal/Documents/M2/grille_processed/297.docx
/home/gcortal/Documents/M2/grille_processed/525.docx
/home/gcortal/Documents/M2/grille_processed/33.docx
/home/gcortal/Documents/M2/grille_processed/249.docx
/home/gcortal/Documents/M2/grille_processed/496.docx
/home/gcortal/Documents/M2/grille_processed/91.docx
/home/gcortal/Documents/M2/grille_processed/356.docx
/home/gcortal/Documents/M2/grille_processed/124.docx
/home/gcortal/Documents/M2/grille_processed/376.docx
/home/gcortal/Documents/M2/grille_processed/74.docx
/home/gcortal/Documents/M2/grille_processed/350.docx
/home/gcortal/Documents/M2/grille_processed/59.docx
/home/gcortal/Documents/M2/grille_processed/28.doc

/home/gcortal/Documents/M2/grille_processed/410.docx
/home/gcortal/Documents/M2/grille_processed/372.docx
/home/gcortal/Documents/M2/grille_processed/390.docx
/home/gcortal/Documents/M2/grille_processed/468.docx
/home/gcortal/Documents/M2/grille_processed/456.docx
/home/gcortal/Documents/M2/grille_processed/449.docx
/home/gcortal/Documents/M2/grille_processed/172.docx
/home/gcortal/Documents/M2/grille_processed/181.docx
/home/gcortal/Documents/M2/grille_processed/482.docx
/home/gcortal/Documents/M2/grille_processed/46.docx
/home/gcortal/Documents/M2/grille_processed/559.docx
/home/gcortal/Documents/M2/grille_processed/19.docx
/home/gcortal/Documents/M2/grille_processed/427.docx
/home/gcortal/Documents/M2/grille_processed/193.docx
/home/gcortal/Documents/M2/grille_processed/512.docx
/home/gcortal/Documents/M2/grille_processed/311.docx
/home/gcortal/Documents/M2/grille_processed/373.docx
/home/gcortal/Documents/M2/grille_processed/84.docx
/home/gcortal/Documents/M2/grille_processed/572.d

/home/gcortal/Documents/M2/grille_processed/447.docx
/home/gcortal/Documents/M2/grille_processed/97.docx
/home/gcortal/Documents/M2/grille_processed/220.docx
/home/gcortal/Documents/M2/grille_processed/585.docx
/home/gcortal/Documents/M2/grille_processed/378.docx
/home/gcortal/Documents/M2/grille_processed/596.docx
/home/gcortal/Documents/M2/grille_processed/144.docx
/home/gcortal/Documents/M2/grille_processed/271.docx
/home/gcortal/Documents/M2/grille_processed/7.docx
/home/gcortal/Documents/M2/grille_processed/96.docx
/home/gcortal/Documents/M2/grille_processed/76.docx
/home/gcortal/Documents/M2/grille_processed/545.docx
/home/gcortal/Documents/M2/grille_processed/494.docx
/home/gcortal/Documents/M2/grille_processed/314.docx
/home/gcortal/Documents/M2/grille_processed/316.docx
/home/gcortal/Documents/M2/grille_processed/198.docx
/home/gcortal/Documents/M2/grille_processed/532.docx
/home/gcortal/Documents/M2/grille_processed/248.docx
/home/gcortal/Documents/M2/grille_processed/219.doc

/home/gcortal/Documents/M2/grille_processed/209.docx
/home/gcortal/Documents/M2/grille_processed/231.docx
/home/gcortal/Documents/M2/grille_processed/265.docx
/home/gcortal/Documents/M2/grille_processed/272.docx
/home/gcortal/Documents/M2/grille_processed/107.docx
/home/gcortal/Documents/M2/grille_processed/52.docx
/home/gcortal/Documents/M2/grille_processed/58.docx
/home/gcortal/Documents/M2/grille_processed/71.docx
/home/gcortal/Documents/M2/grille_processed/347.docx
/home/gcortal/Documents/M2/grille_processed/10.docx
/home/gcortal/Documents/M2/grille_processed/573.docx
/home/gcortal/Documents/M2/grille_processed/389.docx
/home/gcortal/Documents/M2/grille_processed/550.docx
/home/gcortal/Documents/M2/grille_processed/436.docx
/home/gcortal/Documents/M2/grille_processed/236.docx
/home/gcortal/Documents/M2/grille_processed/359.docx
/home/gcortal/Documents/M2/grille_processed/582.docx
/home/gcortal/Documents/M2/grille_processed/551.docx
/home/gcortal/Documents/M2/grille_processed/472.do

In [81]:
grille_df = pd.DataFrame(data, columns = ['name', 'nb_table', 'nb_row', 'nb_col', 'nb_section', 'nb_paragraph', 'text'])

In [82]:
grille_df

,name,nb_table,nb_row,nb_col,nb_section,nb_paragraph,text
0,142.docx,2,7,4,1,6,"[Aéroport lors du passage de la sécurité, la p..."
1,432.docx,2,12,4,1,5,[Vendredi dernier au bureau. Nous sommes 5 dan...
2,168.docx,2,8,2,1,1,[Colère L’examen du permis de conduire. Cet ex...
3,558.docx,4,11,4,1,11,"[Un matin, j’arrive au travail, je croise une ..."
4,481.docx,2,8,4,1,5,"[Un vendredi après-midi, je fais passer un par..."
...,...,...,...,...,...,...,...
593,81.docx,0,0,0,1,25,[]
594,371.docx,2,9,4,1,3,"[Il y a 3 semaines, la fille de mon ami est ve..."
595,147.docx,2,8,2,1,1,[Colère Les faits se sont passés un samedi soi...
596,396.docx,2,11,4,1,5,[Formation Acta. Alain fait un tour de table p...


In [83]:
print(grille_df.to_latex(index=False))  

\begin{tabular}{lrrrrrl}
\toprule
    name &  nb\_table &  nb\_row &  nb\_col &  nb\_section &  nb\_paragraph &                                               text \\
\midrule
142.docx &         2 &       7 &       4 &           1 &             6 & [Aéroport lors du passage de la sécurité, la pe... \\
432.docx &         2 &      12 &       4 &           1 &             5 & [Vendredi dernier au bureau. Nous sommes 5 dans... \\
168.docx &         2 &       8 &       2 &           1 &             1 & [Colère L’examen du permis de conduire. Cet exa... \\
558.docx &         4 &      11 &       4 &           1 &            11 & [Un matin, j’arrive au travail, je croise une c... \\
481.docx &         2 &       8 &       4 &           1 &             5 & [Un vendredi après-midi, je fais passer un part... \\
504.docx &         2 &       8 &       4 &           1 &             5 & [Un lundi matin, j’arrive dans le service de ma... \\
297.docx &         1 &       9 &       4 &           1 &       

## Tokenization, lemmatization

In [407]:
def get_lemma(tokens):
    return [token.lemma_ for token in tokens]

def get_lower(tokens):
    return [token.lower_ for token in tokens]

def get_relevant_tokens(tokens):
    return [token.lower_ for token in tokens if not token.is_stop and not token.is_punct]

def get_relevant_lemma(tokens):
    return [token.lemma_ for token in tokens if not token.is_stop and not token.is_punct]

In [415]:
grille_df = grille_df.explode('text')

In [419]:
grille_df = grille_df.query("text !=''")

In [423]:
grille_df['text'] = grille_df['text'].astype(str)

In [375]:
#grille_df['text_merged'] = grille_df['text'].apply(lambda x: " ".join(x))

In [376]:
#grille_df['tokens'] = grille_df['text_merged'].apply(lambda x: tokenize(x))

In [424]:
grille_df['tokens'] = grille_df['text'].apply(lambda x: tokenize(x))

In [425]:
grille_df['text_lowered'] = grille_df['tokens'].apply(lambda x: get_lower(x))

In [426]:
grille_df['relevant_tokens'] = grille_df['tokens'].apply(lambda x: get_relevant_tokens(x))

In [427]:
grille_df['relevant_lemma'] = grille_df['tokens'].apply(lambda x: get_relevant_lemma(x))

In [428]:
grille_df['lemma'] = grille_df['tokens'].apply(lambda x: get_lemma(x))

In [429]:
grille_df['nb_tokens'] = grille_df['tokens'].apply(lambda x: len(x))

In [430]:
grille_df.query("nb_tokens > 0").describe()

,nb_table,nb_row,nb_col,nb_section,nb_paragraph,nb_tokens
count,1372.000000,1372.000000,1372.000000,1372.000000,1372.000000,1372.000000
mean,3.212099,7.429300,3.552478,1.004373,6.275510,269.749271
std,1.663808,2.551032,0.974968,0.066009,6.220942,219.341372
min,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000
25%,2.000000,6.000000,4.000000,1.000000,3.000000,116.750000
50%,3.000000,8.000000,4.000000,1.000000,5.000000,222.000000
75%,4.000000,9.000000,4.000000,1.000000,8.000000,364.500000
max,8.000000,40.000000,5.000000,2.000000,70.000000,1519.000000


In [499]:
grille_df

,name,nb_table,nb_row,nb_col,nb_section,nb_paragraph,text,tokens,text_lowered,relevant_tokens,relevant_lemma,lemma,nb_tokens
0,142.docx,2,7,4,1,6,"Aéroport lors du passage de la sécurité, la pe...","[Aéroport, lors, du, passage, de, la, sécurité...","[aéroport, lors, du, passage, de, la, sécurité...","[aéroport, passage, sécurité, contrôlant, sacs...","[aéroport, passage, sécurité, contrôler, sac, ...","[aéroport, lors, de, passage, de, le, sécurité...",141
0,142.docx,2,7,4,1,6,Futur de mon parcours scolaire ou professionne...,"[Futur, de, mon, parcours, scolaire, ou, profe...","[futur, de, mon, parcours, scolaire, ou, profe...","[futur, parcours, scolaire, professionnelle, b...","[futur, parcours, scolaire, professionnel, bou...","[futur, de, mon, parcours, scolaire, ou, profe...",128
1,432.docx,2,12,4,1,5,Vendredi dernier au bureau. Nous sommes 5 dans...,"[Vendredi, dernier, au, bureau, ., Nous, somme...","[vendredi, dernier, au, bureau, ., nous, somme...","[vendredi, dernier, bureau, sommes, 5, bureaux...","[vendredi, dernier, bureau, être, 5, bureau, o...","[vendredi, dernier, au, bureau, ., nous, être,...",840
1,432.docx,2,12,4,1,5,"Samedi midi, lors d'une journée de conférences...","[Samedi, midi, ,, lors, d', une, journée, de, ...","[samedi, midi, ,, lors, d', une, journée, de, ...","[samedi, midi, journée, conférences, organisée...","[samedi, midi, journée, conférence, organiser,...","[samedi, midi, ,, lors, de, un, journée, de, c...",330
2,168.docx,2,8,2,1,1,Colère L’examen du permis de conduire. Cet exa...,"[Colère, L’, examen, du, permis, de, conduire,...","[colère, l’, examen, du, permis, de, conduire,...","[colère, examen, permis, conduire, examen, dér...","[colère, examen, permis, conduire, examen, dér...","[colère, l’, examen, de, permis, de, conduire,...",189
...,...,...,...,...,...,...,...,...,...,...,...,...,...
596,396.docx,2,11,4,1,5,Je suis chez moi au téléphone avec monsieur S…...,"[Je, suis, chez, moi, au, téléphone, avec, mon...","[je, suis, chez, moi, au, téléphone, avec, mon...","[téléphone, monsieur, s, directeur, associatio...","[téléphone, Monsieur, S, directeur, associatio...","[je, suivre, chez, moi, au, téléphone, avec, M...",347
597,276.docx,4,5,4,1,2,"Mardi dernier, j’ai retrouvé dans le restauran...","[Mardi, dernier, ,, j’, ai, retrouvé, dans, le...","[mardi, dernier, ,, j’, ai, retrouvé, dans, le...","[mardi, dernier, retrouvé, restaurant, bistrot...","[mardi, dernier, retrouver, restaurant, bistro...","[mardi, dernier, ,, j’, avoir, retrouver, dans...",141
597,276.docx,4,5,4,1,2,"Le weekend dernier, j’ai beaucoup de travail q...","[Le, weekend, dernier, ,, j’, ai, beaucoup, de...","[le, weekend, dernier, ,, j’, ai, beaucoup, de...","[weekend, dernier, beaucoup, travail, greffe, ...","[weekend, dernier, beaucoup, travail, greffe, ...","[le, weekend, dernier, ,, j’, avoir, beaucoup,...",200
597,276.docx,4,5,4,1,2,Je suis dans le fast food tenu par un ami. Il ...,"[Je, suis, dans, le, fast, food, tenu, par, un...","[je, suis, dans, le, fast, food, tenu, par, un...","[fast, food, tenu, ami, neigé, soirée, étudian...","[fast, food, tenir, ami, neiger, soirée, étudi...","[je, suivre, dans, le, fast, food, tenir, par,...",196


In [17]:
grille_df.query("nb_row == 11") #nb_col == 5 # grille_émotion 2013 
grille_df.query("nb_row == 8 & nb_col == 4") # grille_émotion 2014, 2015, 2017 (même format ?)
grille_df.query("nb_row == 7 & nb_col == 4") # grille émotion 2019

,name,nb_table,nb_row,nb_col,nb_section,nb_paragraph,text
0,142.docx,2,7,4,1,6,"[Aéroport lors du passage de la sécurité, la p..."
8,33.docx,1,7,4,1,1,[Mon frère m’a lancé un défi parce qu’il me ju...
21,176.docx,2,7,4,1,7,[Lieu de travail (entreprise) En matinée autou...
36,188.docx,2,7,4,1,4,[Lieu : dans le train Temps : dans la soirée P...
40,2.docx,2,7,4,1,10,"[Lundi, mon collègue ostéo m'annonce qu'il va ..."
...,...,...,...,...,...,...,...
576,137.docx,1,7,4,1,2,[1) 2) 3) Vérifier 1)]
577,32.docx,2,7,4,1,5,[Après deux semaines de travail et la fin du m...
578,203.docx,2,7,4,1,6,"[, ]"
579,16.docx,2,7,4,1,8,"[Dimanche 2 avril, belle journée ensoleillée, ..."


In [66]:
grille_df.query("nb_row == 3 & nb_col == 4") # fait int et ext + vérifications des faits
grille_df.query("nb_row == 0") # pas de tableaux, que du texte
grille_df.query("nb_row == 4") # fait int et ext + besoins à satisfaire. quand nb_col == 2 
#chaque tableau est une grille liée à une émotion
# si nb_col == 5 un tableau est plusieurs grilles avec plusieurs émotions
grille_df.query("nb_row == 6 & nb_col == 4")
grille_df.query("nb_row == 9")

,name,nb_table,nb_row,nb_col,nb_section,nb_paragraph
6,297.docx,1,9,4,1,1
9,249.docx,1,9,4,1,12
13,124.docx,2,9,4,1,3
17,59.docx,2,9,4,1,3
30,269.docx,1,9,4,1,1
...,...,...,...,...,...,...
531,222.docx,1,9,4,1,24
533,146.docx,2,9,4,1,3
590,224.docx,1,9,4,1,1
591,213.docx,1,9,4,1,2


In [153]:
grille_df.query('nb_table == 0') # 30
grille_df.query('nb_table == 1') # 145
grille_df.query('nb_table == 2') # 215
grille_df.query('nb_table == 3') # 55
grille_df.query('nb_table == 4') # 114
grille_df.query('nb_table == 5') # 6
grille_df.query('nb_table == 6') # 26
grille_df.query('nb_table == 7') # 0
grille_df.query('nb_table == 8') # 7

,name,nb_table,nb_row,nb_col,nb_section,nb_paragraph
37,542.docx,8,8,4,1,7
44,597.docx,8,8,4,1,8
114,576.docx,8,8,4,1,8
120,539.docx,8,8,4,1,7
325,596.docx,8,0,0,1,5
336,532.docx,8,8,4,1,8
581,546.docx,8,0,0,1,7


In [106]:
grille_df.query('nb_col == 1') # 0
grille_df.query('nb_col == 2') # 103
grille_df.query('nb_col == 3') # 1
grille_df.query('nb_col == 4') # 455
grille_df.query('nb_col == 5') # 5
grille_df.query('nb_col >= 6') # 0

,name,nb_table,nb_row,nb_col,nb_section,nb_paragraph


In [518]:
Counter(list(itertools.chain.from_iterable(grille_df['relevant_lemma']))).most_common(100)

[('faire', 1336),
 ('colère', 1130),
 ('territoire', 985),
 ('peur', 900),
 ('attaque', 892),
 ('dire', 889),
 ('oui', 825),
 ('x', 804),
 ('voir', 753),
 ('être', 715),
 ('image', 684),
 ('bien', 680),
 ('devoir', 644),
 ('temps', 633),
 ('prendre', 629),
 ('travail', 590),
 ('attaquer', 581),
 ('vouloir', 568),
 ('passer', 553),
 ('savoir', 547),
 ('sentir', 517),
 ('collègue', 516),
 ('non', 491),
 ('venir', 479),
 ('étudiant', 470),
 ('chose', 455),
 ('demande', 453),
 ('intérieur', 432),
 ('aller', 428),
 ('moment', 424),
 ('fois', 424),
 ('rien', 418),
 ('besoin', 408),
 ('cours', 400),
 ('extérieur', 397),
 ('mettre', 397),
 ('répondre', 380),
 ('penser', 380),
 ('émotion', 375),
 ('falloir', 365),
 ('bon', 357),
 ('ressentir', 347),
 ('tristesse', 346),
 ('situation', 344),
 ('avoir', 340),
 ('demander', 336),
 ('question', 332),
 ('tête', 328),
 ('2', 321),
 ('petit', 317),
 ('dernier', 316),
 ('arriver', 312),
 ('pouvoir', 310),
 ('jour', 305),
 ('joie', 304),
 ('heure', 302)

## Topic modelling using Transformers

In [193]:
#!pip install bertopic
#!pip install bertopic[flair]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     |████████████████████████████████| 58 kB 6.6 MB/s eta 0:00:01
     |████████████████████████████████| 28.8 MB 9.5 MB/s eta 0:00:011
     |████████████████████████████████| 5.2 MB 9.1 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 88 kB 9.0 MB/s  eta 0:00:01
     |████████████████████████████████| 79 kB 12.3 MB/s eta 0:00:01
     |████████████████████████████████| 636 kB 10.3 MB/s eta 0:00:01
  Using cached Cython-0.29.28-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
     |███████████████

  Created wheel for hdbscan: filename=hdbscan-0.8.28-cp37-cp37m-linux_x86_64.whl size=3414745 sha256=cbcefd1893e0fedfd7e5925982c91ae5fc2868e6a3f37a92fb322dab5afbafdf
  Stored in directory: /home/gcortal/.cache/pip/wheels/6e/7a/5e/259ccc841c085fc41b99ef4a71e896b62f5161f2bc8a14c97a
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=619e9550f5eae386e3a4224a9dfbeee668642eddee2bd05deaa3a23b50214968
  Stored in directory: /home/gcortal/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82829 sha256=345474e429db34e6c9703cea4f570c7e1d56ecb27582e5a16b7e4d4b2c6ce3b1
  Stored in directory: /home/gcortal/.cache/pip/wheels/b3/52/a5/1fd9e3e76a7ab34f134c07469cd6f16e27ef3a37aeff1fe821
  Created wheel for pynndescent: filename=pynndescent-0.5.6-py3-none-any.whl size=53943 sha256=1c6001a00abcb1329c4d997829eb948b55e30610a782d57446ab590

In [392]:
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop

In [513]:
docs = list()

for sentences in grille_df['text']:
    for sentence in sentences:
        docs.append(sentence)

In [519]:
docs = list()
for doc in list(grille_df['relevant_lemma']):
    docs.append(" ".join(doc))

In [511]:
grille_df.iloc[0]['text']

'Aéroport lors du passage de la sécurité, la personne contrôlant les sacs qui passent le détecteur s’empresse d’avancer le tapis alors que tout est bloqué ce qui amène à faire tomber des affaires Serre les dents, les mains et temperature qui monte mots : Il est con ou il fait exprès (pensées assez fortes car je lui ais poser la question) territoire attaqué : pensées tes affaires par rapport à la situation du au contrôleur le territoire est à moi et l’attaque ext. se contrôler, pensées à s’exprimer pour répondre en gardant son calme Non, j’ai juste réagi très rapidement, je n’ai pas pensé aux différentes actions que j’aurais pu avoir ne pas s’énerver et s’expliquer calmement sur la situation avec le principal intéressé'

In [ ]:
grille_df

In [470]:
print(embeddings.shape)

(1372, 7440)


In [521]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

embeddings = TfidfVectorizer().fit_transform(docs)
topic_model = BERTopic()
topics, probs = topic_model.fit_transform(docs, embeddings)

2022-05-06 13:54:16,428 - BERTopic - Reduced dimensionality
2022-05-06 13:54:16,470 - BERTopic - Clustered reduced embeddings


In [493]:
from bertopic import BERTopic

#topic_model = BERTopic("french")
topic_model = BERTopic(verbose = True, embedding_model="paraphrase-multilingual-MiniLM-L12-v2")
#topic_model = BERTopic(verbose = True, embedding_model="paraphrase-multilingual-mpnet-base-v2")
#topic_model = BERTopic(embedding_model=nlp)
topics, probs = topic_model.fit_transform(docs)

Batches: 100%|██████████| 43/43 [00:28<00:00,  1.50it/s]
2022-05-04 14:53:56,351 - BERTopic - Transformed documents to Embeddings
2022-05-04 14:53:59,377 - BERTopic - Reduced dimensionality
2022-05-04 14:53:59,409 - BERTopic - Clustered reduced embeddings


In [522]:
topic_model.visualize_hierarchy()

In [523]:
#topic_model.reduce_topics(docs, topics, nr_topics=5)

In [524]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,416,-1_faire_colre_territoire_collgue
1,0,64,0_ami_vitrine_rveiller_magasin
2,1,59,1_projet_travail_chef_mail
3,2,58,2_formation_confiance_portefeuille_ok
4,3,55,3_tudiant_examen_enseignant_attaque
5,4,50,4_joie_match_sourire_jean
6,5,43,5_triste_tristesse_sparer_objet
7,6,42,6_mre_fille_mari_prnom
8,7,40,7_prsentation_acteur_mdecin_formation
9,8,37,8_voiture_vhicule_route_chevreuil


In [528]:
topic_model.get_topic(1)

[('projet', 0.04862699958196307),
 ('travail', 0.04565899870052888),
 ('chef', 0.04236858763864747),
 ('mail', 0.029214883800415852),
 ('tableau', 0.024152967921554247),
 ('client', 0.02285909584731468),
 ('faire', 0.02196752592835937),
 ('envoyer', 0.021887630529340715),
 ('vouloir', 0.018411084685622234),
 ('collgue', 0.018354367842886788)]

In [526]:
topic_model.visualize_barchart()

In [527]:
topic_model.visualize_topics()

In [311]:
topic_model.visualize_heatmap()